[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# YOLOv8 Tracking and Counting

---

[![YouTube](https://badges.aleen42.com/src/youtube.svg)](https://youtu.be/OS5qI9YBkfk)
[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow-blogpost.svg)](https://blog.roboflow.com/yolov8-tracking-and-counting/)
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/roboflow/notebooks)

Ultralytics YOLOv8 is the latest version of the YOLO (You Only Look Once) object detection and image segmentation model developed by Ultralytics. The YOLOv8 model is designed to be fast, accurate, and easy to use, making it an excellent choice for a wide range of object detection and image segmentation tasks. It can be trained on large datasets and is capable of running on a variety of hardware platforms, from CPUs to GPUs.

## ⚠️ Disclaimer

YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **23.01.2023** with version **YOLOv8.0.17**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

## Accompanying Blog Post

We recommend that you follow along in this notebook while reading the blog post on how to train YOLOv8 Tracking and Counting, concurrently.

## Pro Tip: Use GPU Acceleration

If you are running this notebook in Google Colab, navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`. This will ensure your notebook uses a GPU, which will significantly speed up model training times.

## Steps in this Tutorial

In this tutorial, we are going to cover:

- Before you start
- Download video
- Install YOLOv8
- Install ByteTrack
- Install Roboflow Supervision
- Tracking utils
- Load pre-trained YOLOv8 model
- Predict and annotate single frame
- Predict and annotate whole video

**Let's begin!**

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Mon Oct  2 07:40:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Download video

In [ ]:
%cd {HOME}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt

/content
^C


In [ ]:
SOURCE_VIDEO_PATH = f"{HOME}/drive/MyDrive/yolov8/pg5.mp4"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **23.01.2023** with version **YOLOv8.0.17**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

In [4]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.190 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


## Install ByteTrack

[ByteTrack](https://github.com/ifzhang/ByteTrack) is great tracker but a bit poorly packaged. We need to jump through some fire hoops to make it work in tandem with [YOLOv8](https://github.com/ultralytics/ultralytics).

In [5]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [6]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from yolox.tracker.basetrack import BaseTrack
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 50
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

## Install Roboflow Supervision

In [7]:
#!pip install supervision==0.1.0
!pip install supervision==0.14.0

from IPython import display
display.clear_output()


import supervision
import supervision as sv
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.14.0


In [8]:
from supervision.draw.color import ColorPalette
from supervision.geometry.core import Point
from supervision.utils.video import VideoInfo
from supervision.utils.video import get_video_frames_generator
from supervision.utils.video import VideoSink
#from supervision.notebook.utils import show_frame_in_notebook
from supervision.detection.annotate import Detections, BoxAnnotator
#from supervision.detection.line_counter import LineCounterAnnotator



## Tracking utils

Unfortunately, we have to manually match the bounding boxes coming from our model with those created by the tracker.

In [9]:
from typing import List

import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id
    #print("here is in tracker functions id ",tracks)

    return tracker_ids

## Load pre-trained YOLOv8 model

In [10]:
# settings
MODEL = "/content/drive/MyDrive/yolov8/last.pt"

In [11]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


## Predict and annotate single frame

In [12]:
SOURCE_VIDEO_PATH = f"{HOME}/drive/MyDrive/yolov8/pg5.mp4"

In [13]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - car, motorcycle, bus and truck (for me pig,head,tail)
CLASS_ID = [0]

In [17]:
!pip install opencv-python-headless


In [19]:
import random
import cv2
from supervision.geometry.core import Point
from supervision.geometry.core import Tuple
supervision.utils.notebook.plot_image
COLORS = sv.ColorPalette.default()



##ROI
polygon = np.array([[1634, 518],[1386, 522],[1382, 618],[1630, 614],[1630, 518]])
# Create a Polygon object
#polygon = cv2.convexHull(polygon)

##################

#np.array([[1529, 886],[1593, 886],[1589, 818],[1525, 818],[1525, 886]])
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

#read next_id from BaseTrack
n_id = BaseTrack()
print("read next id from BaseTrack =",n_id.next_id())

# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
# create instance of BoxAnnotator
byte_tracker = BYTETracker(BYTETrackerArgs())
box_annotator = BoxAnnotator(color=COLORS, thickness=4, text_thickness=4, text_scale=2)

zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.red(), thickness=6)

# acquire first video frame
iterator = iter(generator)
frame = next(iterator)

#######ROI
area_roi = [np.array([ (1529, 886),(1593, 886),(1589, 818) ,(1525, 818)], np.int32)]
ROI = frame[818:886, 1525:1593]
##########

# model prediction on single frame and conversion to supervision Detections
results = model(frame)
detections = sv.Detections(
    xyxy=results[0].boxes.xyxy.cpu().numpy(),
    confidence=results[0].boxes.conf.cpu().numpy(),
    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
)

#ROI
def is_point_inside_polygon(point, polygon):
    # Check if a point is inside a polygon using the "ray casting" algorithm
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def get_unique_identifier(detection, polygon):
    if (
        len(detection) >= 4
        and detection[0] is not None
        and detection[1] is  None
        and detection[2] is not None
        and detection[3] is not None
    ):
        center_x = (detection[0][0] + detection[0][2]) / 2
        center_y = (detection[0][1] + detection[0][3]) / 2
        class_id = detection[4]

        unique_identifier = f"{center_x}_{center_y}_{class_id}"

        # Check if the center of the detection is inside the polygon
        if is_point_inside_polygon((center_x, center_y), polygon):
            x = (class_id, "found in ROI ")
            return x
            #return unique_identifier
        else:
            print("Detection is outside the polygon")
            return None
    else:
        # Handle the case where the detection doesn't have enough elements or contains None values
        #print("Invalid detection format:", detection)
        return None

#######################

# filtering out detections with unwanted classes
mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
#detections.filter(mask=mask, inplace=True)

# tracking detections
tracks = byte_tracker.update(
    output_results=detections2boxes(detections=detections),
    img_info=frame.shape,
    img_size=frame.shape
)


tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
#print("detectioons tracker id = ",detections.tracker_id,'\n')
#print("tracker id = ",tracker_id,'\n')

lost_id = [0]
detections.tracker_id = np.array(tracker_id)

# format custom labels
'''labels = [
            f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} "
            for _, confidence, class_id, tracker_id
            in detections
]'''
labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]

# annotate and display frame
#frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
frame = zone_annotator.annotate(scene=frame)
t = 0

for detection in detections:
    unique_identifier = get_unique_identifier(detection, polygon)
    if unique_identifier:
        print(t," = Unique Identifier:", unique_identifier)
        t+=1
%matplotlib inline
#show_frame_in_notebook(frame, (16, 16))
sv.plot_image(frame,size=(16, 16))

DisabledFunctionError: ignored

## Predict and annotate whole video

In [14]:
# settings

TARGET_VIDEO_PATH = f"{HOME}/drive/MyDrive/yolov8/res27.mp4"

In [15]:
SOURCE_VIDEO_PATH = f"{HOME}/drive/MyDrive/yolov8/pg5.mp4"

In [16]:
VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=1920, height=1080, fps=30, total_frames=833)

In [ ]:
from tqdm.notebook import tqdm
import random
import cv2
from supervision.geometry.core import Point
from supervision.geometry.core import Tuple
supervision.utils.notebook.plot_image
COLORS = sv.ColorPalette.default()
cap = cv2.VideoCapture(0)
@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 50
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False
from supervision.draw.color import ColorPalette
from supervision.geometry.core import Point
from supervision.utils.video import VideoInfo
from supervision.utils.video import get_video_frames_generator
from supervision.utils.video import VideoSink
#from supervision.notebook.utils import show_frame_in_notebook
from supervision.detection.annotate import Detections, BoxAnnotator
#from supervision.detection.line_counter import LineCounterAnnotator

# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(color=COLORS, thickness=2, text_thickness=1, text_scale=1)
#########ROI
polygon = np.array([[1634, 518],[1386, 522],[1382, 618],[1630, 614],[1630, 518]])
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.blue(), thickness=6)
################
tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)

#variables for me
pre_tracker_id = [0]
lost_id = [0]

##################ROI
#polygon = np.array([[1634, 518],[1386, 522],[1382, 618],[1630, 614],[1630, 518]])
#polygon = np.array([[1529, 886],[1593, 886],[1589, 818],[1525, 818],[1525, 886]])
#polygon = np.array([[1393, 902],[1389, 802],[1613, 802],[1613, 898],[1389, 898]])
#polygon = np.array([[1265, 810],[1245, 250],[1561, 246],[1585, 802],[1265, 810]])
#product randm number, later for read number from scanner

#######################ROI
def is_point_inside_polygon(point, polygon):
    # Check if a point is inside a polygon using the "ray casting" algorithm
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def get_unique_identifier(detection, polygon):
    if (
        len(detection) >= 4
        and detection[0] is not None
        and detection[1] is  None
        and detection[2] is not None
        and detection[3] is not None
    ):
        center_x = (detection[0][0] + detection[0][2]) / 2
        center_y = (detection[0][1] + detection[0][3]) / 2
        class_id = detection[4]

        unique_identifier = f"{center_x}_{center_y}_{class_id}"

        # Check if the center of the detection is inside the polygon
        if is_point_inside_polygon((center_x, center_y), polygon):
            x = (class_id, "found in ROI ")

            ###Changin Tracking ID
            updated_tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
            )
            target_track_id_to_change = class_id

            for track in updated_tracks:
              # Check if the current track has the target tracking ID
              if track.track_id == target_track_id_to_change:
                  # Modify the tracking ID to the new value (Read from scanner)
                  track.track_id = "scanned"  # Replace with your desired new tracking ID
                  continue
                #track.track_id = "scanned"
            return x #return unique_identifier

        else:
            #print("Detection is outside the polygon")
            return None
    else:
        # Handle the case where the detection doesn't have enough elements or contains None values
        #print("Invalid detection format:", detection)
        return None

############################

def product_num():
  num = random.randrange(200,300)
  return num


# open target video file
with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # loop over video frames
    for frame in tqdm(generator, total=video_info.total_frames):
        # model prediction on single frame and conversion to supervision Detections
        results = model(frame)[0]
        detections = Detections(
            xyxy=results.boxes.xyxy.cpu().numpy(),
            confidence=results.boxes.conf.cpu().numpy(),
            class_id=results.boxes.cls.cpu().numpy().astype(int)
        )
        # filtering out detections with unwanted classes
        mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        #detections.filter(mask=mask, inplace=True)
        detections = detections[detections.class_id == 0]

        #Every 'run' the id numbers are increasing
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )

        #print(tracker_id)
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)

        #print(len(pre_tracker_id)," previous ID's  =",pre_tracker_id)
        #print(len(tracker_id)," Current ID's  = ",tracker_id)

        #Checking every Tracker ID's list with previous ID's list
        i = (int)
        for i in range(0, len(pre_tracker_id)):
          idflag = False
          for j in range(0,len(tracker_id)):
            #print("if",pre_tracker_id[i]," != ", tracker_id[j])
            if(pre_tracker_id[i]!= tracker_id[j]):
              idflag = True
            else:
              idflag = False
              break
          if idflag == True:
            lost_id.append(pre_tracker_id[i])

        #print("lost ids = ",lost_id)
        lost_id = [0]

        '''for i in range(0,len(detections.tracker_id)):
          #print("if ",tracker_id[i],"==",'None')
          if(detections.tracker_id[i] == None):
            detections.tracker_id[i]= product_num()
            #print("tracker ID ",i," = ",tracker_id[i])
        '''
        t = 0
        ##############ROI
        for detection in detections:
            unique_identifier = get_unique_identifier(detection, polygon)
            if unique_identifier:
                print(t," = Inside the area ", unique_identifier)
                t+=1
        ###################

        '''labels = [
            f"#{tracker_id} {CLASS_NAMES_DICT[class_id]}"
            for _, confidence, class_id, tracker_id
            in detections
        ]'''
        labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]
        #to check same id or not
        pre_tracker_id = tracker_id
        # annotate and display frame
        frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
        frame = zone_annotator.annotate(scene=frame)
        sink.write_frame(frame)

  0%|          | 0/5277 [00:00<?, ?it/s]


0: 384x640 17 pigs, 12 heads, 11 tails, 20.9ms
Speed: 3.1ms preprocess, 20.9ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 pigs, 12 heads, 11 tails, 24.3ms
Speed: 2.9ms preprocess, 24.3ms inference, 6.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 pigs, 11 heads, 12 tails, 6.2ms
Speed: 2.4ms preprocess, 6.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


0  = Inside the area  (None, 'found in ROI ')
1  = Inside the area  (None, 'found in ROI ')
0  = Inside the area  (None, 'found in ROI ')
1  = Inside the area  (None, 'found in ROI ')
0  = Inside the area  (None, 'found in ROI ')
1  = Inside the area  (None, 'found in ROI ')



0: 384x640 16 pigs, 13 heads, 11 tails, 7.0ms
Speed: 3.3ms preprocess, 7.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 pigs, 12 heads, 11 tails, 6.7ms
Speed: 2.8ms preprocess, 6.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 pigs, 12 heads, 11 tails, 6.2ms
Speed: 3.0ms preprocess, 6.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 pigs, 14 heads, 12 tails, 12.0ms
Speed: 2.9ms preprocess, 12.0ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)



0  = Inside the area  (1, 'found in ROI ')
1  = Inside the area  (None, 'found in ROI ')
0  = Inside the area  ('scanned', 'found in ROI ')
1  = Inside the area  (None, 'found in ROI ')
0  = Inside the area  ('scanned', 'found in ROI ')
1  = Inside the area  (None, 'found in ROI ')
0  = Inside the area  ('scanned', 'found in ROI ')
1  = Inside the area  (None, 'found in ROI ')


...

0: 384x640 13 pigs, 11 heads, 12 tails, 9.8ms
Speed: 2.6ms preprocess, 9.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 pigs, 11 heads, 12 tails, 9.2ms
Speed: 3.9ms preprocess, 9.2ms inference, 5.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 pigs, 11 heads, 12 tails, 8.7ms
Speed: 2.7ms preprocess, 8.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 pigs, 11 heads, 12 tails, 9.0ms
Speed: 2.8ms preprocess, 9.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 pigs, 11 heads, 12 tails, 8.0ms
Speed: 2.7ms preprocess, 8.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 pigs, 11 heads, 12 tails, 8.1ms
Speed: 2.9ms preprocess, 8.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 pigs, 12 heads, 13 tails, 8.4ms
Speed: 2.9ms preprocess, 8.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)


0  = Inside the area  ('scanned', 'found in ROI ')
0  = Inside the area  ('scanned', 'found in ROI ')
0  = Inside the area  ('scanned', 'found in ROI ')



0: 384x640 13 pigs, 11 heads, 12 tails, 9.9ms
Speed: 2.8ms preprocess, 9.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 pigs, 12 heads, 12 tails, 9.8ms
Speed: 4.8ms preprocess, 9.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 pigs, 12 heads, 12 tails, 9.5ms
Speed: 2.7ms preprocess, 9.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 pigs, 11 heads, 12 tails, 9.0ms
Speed: 2.7ms preprocess, 9.0ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 pigs, 12 heads, 12 tails, 8.6ms
Speed: 2.8ms preprocess, 8.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 pigs, 12 heads, 12 tails, 8.6ms
Speed: 2.7ms preprocess, 8.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 pigs, 12 heads, 13 tails, 8.9ms
Speed: 2.8ms preprocess, 8.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0  = Inside the area  ('scanned', 'found in ROI ')
0  = Inside the area  ('scanned', 'found in ROI ')



0: 384x640 14 pigs, 12 heads, 11 tails, 8.8ms
Speed: 2.8ms preprocess, 8.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 pigs, 12 heads, 11 tails, 8.5ms
Speed: 2.7ms preprocess, 8.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 pigs, 11 heads, 11 tails, 8.4ms
Speed: 2.9ms preprocess, 8.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 pigs, 11 heads, 11 tails, 8.7ms
Speed: 2.9ms preprocess, 8.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 pigs, 9 heads, 11 tails, 8.8ms
Speed: 2.8ms preprocess, 8.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 pigs, 8 heads, 11 tails, 10.1ms
Speed: 2.8ms preprocess, 10.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 pigs, 9 heads, 12 tails, 7.7ms
Speed: 2.7ms preprocess, 7.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)


## 🏆 Congratulations

### Learning Resources

Roboflow has produced many resources that you may find interesting as you advance your knowledge of computer vision:

- [Roboflow Notebooks](https://github.com/roboflow/notebooks): A repository of over 20 notebooks that walk through how to train custom models with a range of model types, from YOLOv7 to SegFormer.
- [Roboflow YouTube](https://www.youtube.com/c/Roboflow): Our library of videos featuring deep dives into the latest in computer vision, detailed tutorials that accompany our notebooks, and more.
- [Roboflow Discuss](https://discuss.roboflow.com/): Have a question about how to do something on Roboflow? Ask your question on our discussion forum.
- [Roboflow Models](https://roboflow.com): Learn about state-of-the-art models and their performance. Find links and tutorials to guide your learning.

### Convert data formats

Roboflow provides free utilities to convert data between dozens of popular computer vision formats. Check out [Roboflow Formats](https://roboflow.com/formats) to find tutorials on how to convert data between formats in a few clicks.

### Connect computer vision to your project logic

[Roboflow Templates](https://roboflow.com/templates) is a public gallery of code snippets that you can use to connect computer vision to your project logic. Code snippets range from sending emails after inference to measuring object distance between detections.